In [1]:
import random 
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
transition_matrix = pd.read_csv('./data/transition_matrices/transition_matrix_monday.csv', index_col=0)
transition_matrix

,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.089843,0.743920,0.062375,0.051502,0.052361
drinks,0.208109,0.011236,0.610650,0.090865,0.079140
fruit,0.206411,0.087959,0.050821,0.607115,0.047694
spices,0.145765,0.191368,0.176710,0.096906,0.389251


In [3]:
class Customer:
    """
    a single customer that moves through the supermarket
    in a MCMC simulation
    """
    def __init__(self, name, state, budget=100):
        self.name = name
        self.state = state
        self.budget = budget
    def __repr__(self):
        return f'customer_no {self.name} {self.state}'
    def next_state(self):
        '''
        Propagates the customer to the next state.
        Returns nothing.
        '''
        self.state = random.choices(['checkout', 'dairy', 'drinks', 'fruit', 'spices'], list(transition_matrix.loc[self.state]) )
        self.state = self.state[0]
        
        
    def is_active(self):
        
        if self.state == 'checkout':
            return True
        

In [4]:
class Supermarket:
    """manages multiple Customer instances that are currently in the market."""

    def __init__(self,market_name,opening,closing): 
        self.market_name = market_name
        self.opening = opening
        self.closing = closing
        self.customers = []
        self.current_time = 0
        self.index=0
        self.customer_index =0
        self.state = None
        self.dti = pd.date_range(self.opening, self.closing, freq="T").time
        
        
    def __repr__(self):
        return f'{sefl.market_name}'
    
    def is_open(self):
        if self.index <= len(self.dti)-2:
            return datetime.strptime(self.opening, '%H:%M:%S') <= datetime.strptime(self.get_time(), '%H:%M:%S') <= datetime.strptime(self.closing, '%H:%M:%S')

    def get_time(self):
        """current time in HH:MM format,"""
        self.current_time = str(self.dti[self.index])
        return self.current_time
        
    def print_customers(self):
        """print all customers with the current time and id in CSV format.
        """
        return self.customers

    def next_minute(self):
        """"propagates all customers to the next state."""
        
        self.index += 1
        next_time = self.dti[self.index] 
        
        for customer in self.customers:
            customer.next_state()

            
    def add_new_customers(self):
        """randomly creates new customers.
        """
        self.state = random.choices(['dairy', 'drinks', 'fruit', 'spices'])
        self.state = self.state[0]
        self.customer_index +=1
        new_customer = Customer(self.customer_index,self.state)
        self.customers.append(new_customer)
        

    def remove_exitsting_customers(self):
        """removes every customer that is not active any more.
        """
        for customer in self.customers:
            if customer.is_active():
                self.customers.remove(customer)

In [5]:
#initialization for the while-loop
a = Supermarket('MarketRangers','07:00:00','22:00:00')
market_rangers = pd.DataFrame()

while a.is_open():    
    a.add_new_customers()
    for customer in a.customers:
        market_rangers = market_rangers.append({'time': a.get_time(), 
                                                'customer_no': str(customer.name),
                                                'location': customer.state
                                               },ignore_index=True)
    
    a.remove_exitsting_customers()
    a.next_minute()
    
market_rangers.set_index('time', inplace=True)
market_rangers.head(20)   

,customer_no,location
time,,
07:00:00,1,fruit
07:01:00,1,fruit
07:01:00,2,fruit
07:02:00,1,drinks
07:02:00,2,dairy
07:02:00,3,spices
07:03:00,1,drinks
07:03:00,2,spices
07:03:00,3,fruit


In [6]:
market_rangers.to_csv('./data/Simulated_Market_Table/simulated_market_table_monday.csv')